# Tarea de la Semana 3:<br>Buena Arquitectura de Datos

¡Bienvenido al laboratorio de la semana 3! En este laboratorio, explorarás y evaluarás los aspectos de seguridad, rendimiento, confiabilidad y escalabilidad de una aplicación web alojada en AWS. Harás esto simulando tráfico hacia tu aplicación web y utilizando herramientas de AWS como Amazon CloudWatch para monitorear los recursos informáticos y la actividad de red en tu aplicación web. Configurarás los recursos informáticos para habilitar la eficiencia de rendimiento, así como opciones de seguridad para controlar el tráfico entrante a tu aplicación web. Realizarás estas tareas a través de los principios de una "buena" arquitectura de datos, así como a través de las lentes del Marco Bien Arquitectado de AWS. Al pasar por este laboratorio, obtendrás una comprensión de los recursos primitivos disponibles en AWS, para que luego puedas aplicar esta comprensión al diseñar tus canalizaciones de datos como Ingeniero de Datos.

# Tabla de Contenidos
- [1 - Introducción](#1)
- [2 - Obteniendo la Dirección de la Aplicación Web](#2)
- [3 - Monitoreo del Uso de CPU y Actividad de Red](#3)
- [4 - Mejorando la Seguridad](#4)
- [5 - Verificando la Disponibilidad de EC2](#5)
- [6 - Realizando Escalado Automático](#6)
    - [6.1 - Utilizando Recursos de Manera Eficiente](#6-1)
    - [6.2 - Realizando Escalado Automático](#6-2)

---


<div id='1'/>

## 1 - Introducción

En la semana 2, implementaste un pipeline de datos donde ingresaste y transformaste datos y luego los serviste a un analista de datos.

![image alt ><](./images/ETL.drawio.png)

Ahora supongamos que construiste un pipeline de datos similar para servir los datos transformados a un público más amplio. Por ejemplo, supongamos que tu empresa vende algunos datos analíticos y los pone a disposición de personas en todo internet. O supongamos que necesita servir algunos paneles incrustados a sus clientes. Para hacerlo, necesitarás construir una aplicación web que ponga los datos o paneles a disposición de sus clientes previstos y que sea capaz de escalar según sus necesidades. En este laboratorio, te enfocarás en este componente de tu arquitectura de datos (la aplicación web) para evaluar su seguridad, rendimiento, confiabilidad y escalabilidad.

¿Cómo se crea la aplicación web? Una forma típica de implementar soluciones de aplicaciones web es utilizar una [arquitectura de tres capas](https://docs.aws.amazon.com/whitepapers/latest/serverless-multi-tier-architectures-api-gateway-lambda/three-tier-architecture-overview.html) que consta de tres capas o niveles:


- **Capa de presentación**: esta capa representa la interfaz de usuario del sitio web (por ejemplo, una página web) que permite a los clientes interactuar con la aplicación web utilizando sus dispositivos. Aquí es donde se pueden mostrar los paneles analíticos para los clientes.
- **Capa lógica**: esta capa, también conocida como capa de aplicación, representa la lógica empresarial que procesa la entrada de los clientes, realiza consultas a los almacenes de datos internos y devuelve los resultados que deben mostrarse en la capa de presentación.
- **Capa de datos**: esta capa es donde se almacenan los datos asociados con la aplicación web.

Aquí se te presenta la arquitectura de tres capas de tu aplicación web, y principalmente interactuarás con los recursos informáticos en los que se ejecuta la lógica de la aplicación. Aquí tienes el diagrama arquitectónico de la aplicación web:


En el lado izquierdo, se ve S3 que representa el almacén de datos de la capa de datos, solo como un ejemplo de una capa de datos. En el lado derecho, se ven los clientes que interactúan con el sitio web a través de sus dispositivos. Los otros dos componentes principales son **Application Load Balancer (ALB)** y **Auto Scaling group** que operan en la capa de aplicación:


- [Grupo de escalado automático de Amazon EC2](https://docs.aws.amazon.com/autoscaling/ec2/userguide/what-is-amazon-ec2-auto-scaling.html): este grupo consiste en una colección de instancias de EC2. ¿Qué es una instancia de EC2? Este es el servidor de aplicaciones en el que se ejecuta la lógica de tu aplicación. Principalmente consiste en la Unidad Central de Procesamiento (CPU) y la Memoria de Acceso Aleatorio (RAM). Piensa en EC2 como una computadora proporcionada por AWS en la que ejecutarás tu código. ¿Por qué tendrías más de una EC2? Cada EC2 ejecuta la misma lógica y se utilizan para aumentar las capacidades informáticas de tu aplicación. Entonces, en lugar de tener una sola EC2 que procesa todas las entradas o solicitudes de los clientes, estas entradas se distribuyen entre las instancias de EC2. **Escalado automático** significa que el número de instancias de EC2 puede aumentar o disminuir según la demanda. En este caso, se trata de las entradas o solicitudes de los clientes a la aplicación web.

- [Balanceador de carga de aplicaciones](https://docs.aws.amazon.com/elasticloadbalancing/latest/userguide/what-is-load-balancing.html): El grupo de escalado automático está asociado con un balanceador de carga de aplicaciones, que distribuye el tráfico de aplicaciones entrante (solicitudes de clientes o entradas) entre las instancias de EC2. El balanceador de carga sirve como el único punto de contacto para los clientes.

En el diagrama, también se ven dos términos: **Red Privada Virtual (VPC)** y **subred**. VPC es una forma de aislar tus recursos (por ejemplo, EC2) del mundo exterior. Piénsalo como una caja o una pared que protege tus recursos, y también como una forma de organizarlos. Los recursos dentro de la VPC pueden comunicarse entre sí. Pero por defecto, no hay comunicación entre la VPC y el internet exterior a menos que permitas que esta comunicación ocurra configurando adecuadamente la VPC. Ahora, dentro de tu VPC, es posible que necesites que algunos recursos sean públicos y que otros sean privados. ¿Cómo puedes hacer eso? Este es el papel de las subredes que puedes crear dentro de tu VPC. Las subredes te proporcionan un control más detallado sobre el acceso a tus recursos. Puedes crear una **subred pública** si deseas permitir que el tráfico exterior acceda a tus recursos, y puedes crear una **subred privada** si no deseas permitir que el tráfico exterior acceda a tus recursos.


<div id='2'/>

## 2 - Obteniendo la dirección de la aplicación web

La arquitectura de la aplicación web está implementada y se proporciona en este laboratorio. Tu primera tarea es obtener la dirección de tu aplicación web. Para hacerlo, necesitas encontrar el ALB, ya que sirve como el único punto de contacto para los clientes.


2.1. Para acceder a la consola de AWS, ejecuta el siguiente comando en la terminal.
Si es necesario, abre una nueva terminal seleccionando Terminal > Nueva terminal en el menú.

```bash
cat ../.aws/aws_console_url
```
Abre el enlace en una nueva ventana del navegador.

*Nota*: Por razones de seguridad, la URL para acceder a la consola de AWS caducará cada 15 minutos, 
pero cualquier recurso de AWS que hayas creado seguirá disponible durante el período de 2 horas. 
Si necesitas acceder a la consola después de 15 minutos, vuelve a ejecutar el comando para obtener un nuevo enlace activo.

*Nota:* Si ves la ventana como en la siguiente captura de pantalla, haz clic en el enlace **cerrar sesión**, 
cierra la ventana y haz clic en el enlace de la consola nuevamente.

![AWSLogout](images/AWSLogout.png)


2.2. Para encontrar el Balanceador de Carga de Aplicaciones (ALB) que se ha creado como parte de la arquitectura de tu aplicación, ve a la consola de AWS y busca **EC2**.

![imagen alt ><](./images/EC2.png)


2.3. En el panel izquierdo, haz clic en **Balanceadores de carga** (parte inferior del panel)

![image alt ><](./images/LoadBalancers.png)


2.4. Verás un balanceador de carga llamado `de-c1w3-alb`. Copia el campo `Nombre DNS`. DNS significa Sistema de Nombres de Dominio que traduce nombres de dominio como ejemplo.com en una dirección IP 
(puedes leer más sobre DNS [aquí](https://aws.amazon.com/route53/what-is-dns/)).

![imagen alt ><](./images/LoadBalancerDNS.png)


2.5. Abre una nueva ventana del navegador y pega el Nombre DNS. Verás el siguiente mensaje mostrado en la página web.

*Nota*: Después de copiar el nombre DNS en el paso 2.4 y pegarlo en una nueva pestaña del navegador, es posible que te encuentres con un error de "Este sitio no puede ser alcanzado".
Verifica si el navegador está intentando usar el protocolo HTTPS (`https://`). Si es así, cambia manualmente la URL para utilizar en su lugar el protocolo HTTP (`http://`).

![imagen alt ><](./images/DashboardOriginal.png)

La página web debería mostrar un panel de control para clientes, pero por brevedad y simplicidad, se te muestra un mensaje simple. No te preocupes por los detalles de este mensaje, 
los explorarás en una sección posterior de este laboratorio. Al abrir la página web, enviaste una solicitud HTTP al ALB. La solicitud HTTP llegó al ALB a través del puerto 80, 
que es el [puerto predeterminado](https://www.techopedia.com/definition/15709/port-80#:~:text=To%20make%20it%20convenient%20for,Port%2080%20should%20be%20used) para solicitudes HTTP 
(si no sabes qué significa puerto, consulta [aquí](https://www.cloudflare.com/learning/network-layer/what-is-a-computer-port/)).

En las siguientes secciones, aprenderás cómo monitorear la actividad informática y de red de tu aplicación web y te asegurarás de que los principios de una arquitectura de datos "buena" estén en su lugar. 
En particular, configurarás tu arquitectura para abrazar los siguientes principios: "Priorizar la seguridad", "Planificar para el fallo" y "Arquitecturar para la escalabilidad".


## 3 - Monitoreo del uso de CPU y actividad de red

Cuando compartes tu aplicación web con tus clientes, debes esperar un cierto tráfico entrante a tu aplicación que necesita ser procesado. Para asegurarte de que tu aplicación pueda 
soportar las demandas entrantes, necesitarías monitorear el uso de los recursos informáticos de tu aplicación web. Para hacerlo, puedes utilizar Amazon CloudWatch, que es el servicio de monitoreo 
y observabilidad en AWS que te permite recopilar y rastrear métricas de cómputo y memoria para tu aplicación web.

Monitorear el uso de los recursos informáticos de tu aplicación web es una de las prácticas del subyacente DataOps y del pilar de Excelencia Operativa del 
[Marco de trabajo bien arquitectado de AWS](https://aws.amazon.com/architecture/well-architected/?wa-lens-whitepapers.sort-by=item.additionalFields.sortDate&wa-lens-whitepapers.sort-order=desc&wa-guidance-whitepapers.sort-by=item.additionalFields.sortDate&wa-guidance-whitepapers.sort-order=desc). 
Estas prácticas enfatizan la importancia de integrar la automatización, el monitoreo de sistemas y la adaptabilidad en el diseño de cargas de trabajo en la nube. Estas prácticas permiten comprender 
tus cargas de trabajo y sus comportamientos anticipados.

En esta sección, vas a realizar algunas pruebas de estrés para simular tráfico a tu sitio web y luego monitorear el uso de CPU y la actividad de red utilizando Amazon CloudWatch.


3.1. Para realizar pruebas de estrés sobre su arquitectura actual, utilizará una herramienta de evaluación de código abierto llamada [Apache Benchmark](https://httpd.apache.org/docs/2.4/programs/ab.html).
Esta herramienta se utiliza para probar la capacidad de un sitio web para manejar un gran número de solicitudes HTTP al mismo tiempo.

En la consola de AWS, busque el servicio **CloudShell**:

![image alt ><](./images/CloudShell.png)

AWS CloudShell es un shell basado en el navegador que facilita la gestión, exploración e interacción segura con sus recursos de AWS.


3.2. Instalar la herramienta Apache Benchmark con el comando de terminal (la opción `-y` permite confirmar automáticamente la instalación del paquete sin intervención manual):

```bash
sudo yum install httpd-tools -y
```


3.3. Para generar una prueba de estrés en tu servidor, necesitarás realizar múltiples solicitudes HTTP GET con cierta concurrencia, es decir, que las solicitudes se ejecuten al mismo tiempo.
Esto se puede hacer con el comando `ab` de Apache Benchmark.
Ejecuta el siguiente comando, reemplazando `ALB-DNS` con el `Nombre DNS` de tu ALB (debería estar en la dirección de la ventana del navegador donde viste la representación del panel de control).

```bash
ab -n 7000 -c 50 http://<ALB-DNS>/
```

Asegúrate de agregar la ruta `/` al final. Las opciones utilizadas en el comando se utilizan con los siguientes propósitos:

- La opción `-n` corresponde al número total de solicitudes que se enviarán al servidor HTTP.
- La opción `-c` corresponde al número de solicitudes concurrentes (simultáneas) que se realizarán.


3.4. Para monitorear el uso de la CPU y la red, ve a los **Grupos de escalado automático** en la consola de AWS (es parte de la sección de EC2):

![image alt ><](./images/AutoScalingGroups.png)


3.5. Selecciona el grupo de escalado automático creado para ti y selecciona la pestaña de **Monitoreo** y luego haz clic en **EC2**:

![image alt ><](./images/Monitoring.png)

Allí puedes ver algunas de las métricas disponibles que se miden, que incluyen CPU, Red (entrante y saliente), entre otros. 
Durante la prueba, puedes verificar los gráficos de CPU y Red para ver un aumento en las métricas; las actualizaciones de los gráficos pueden tardar un tiempo ya que las métricas en AWS CloudWatch tienen una latencia de 5-10 minutos. 
Necesitarás hacer clic en el botón ![](./images/Refresh.png) para ver las actualizaciones.
Hay otras métricas disponibles y la posibilidad de definir nuevas a través de AWS CloudWatch.

AWS CloudWatch es el servicio de monitoreo y observabilidad en AWS que permite a los usuarios recopilar y rastrear métricas, recopilar y monitorear archivos de registro, y establecer alarmas. 
Está diseñado para proporcionar una vista integral de los recursos de AWS, aplicaciones y servicios que se ejecutan en la infraestructura de AWS.

Después de que se realicen todas las solicitudes, deberías notar una reducción en la actividad de la CPU, así como en la red.


<div id='4'/>

## 4 - Mejorando la seguridad

Ahora que sabes cómo monitorear los recursos informáticos de tu aplicación, es hora de adentrarte en el aspecto de seguridad de tus servidores. La seguridad siempre debe ser tu prioridad, como se establece en el principio "priorizar la seguridad". Además, el pilar de seguridad proporciona información sobre cómo aprovechar las tecnologías en la nube para asegurar datos, sistemas y activos, y protegerlos de accesos no autorizados.

Te han informado que a través del puerto 90 del ALB se muestra ciertos datos privados, pero solo ciertas personas dentro de tu empresa deberían tener acceso a ellos. En esta sección, resolverás este problema para que cualquier tráfico entrante solo pueda pasar a través del puerto 80. En particular, trabajarás ajustando los **grupos de seguridad** del ALB, que es una herramienta que actúa como un cortafuegos virtual, controlando el tráfico entrante y saliente a través del ALB.

4.1. Toma el `Nombre DNS` de tu ALB y pégalo en la barra de búsqueda de tu navegador apuntando al puerto 90, de la siguiente manera:

```
<DNS-ALB>:90
```

A través de este puerto, verás un mensaje que imita la visualización de algunos datos privados. Te sorprende dado los requisitos de seguridad que deberían haber sido implementados, así que decides solucionarlo ahora.


4.2. En la consola de AWS, nuevamente buscas **EC2** y luego haces clic en **Grupos de seguridad** en el panel izquierdo.

![image alt ><](./images/SecurityGroups.png)

Verás una lista de grupos de seguridad, con los cuales trabajarás con los siguientes:
- `de-c1w3-ec2-sg`: Este es el grupo de seguridad de tus instancias EC2 (servidores).
- `de-c1w3-alb-sg`: Este es el grupo de seguridad del Balanceador de Carga.

4.3. Haz clic en el id del grupo de seguridad con el nombre `de-c1w3-ec2-sg`. En la sección de **Reglas de entrada**, encontrarás la siguiente regla:

- Regla con `Tipo: Todo TCP`, `Rango de puertos: 0-65535` y como origen un id de grupo de seguridad. Este origen es el grupo de seguridad del ALB. El propósito de este grupo de seguridad es asegurar que los servidores solo recibirán tráfico del ALB.

4.4. Regresa y haz clic en el ID del grupo de seguridad `de-c1w3-alb-sg`. Verifica las **Reglas de entrada** para encontrar la siguiente regla:

- Regla con `Tipo: Todo TCP`, `Rango de puertos: 0-65535` y `Origen: 0.0.0.0/0` (lo que significa todas las direcciones IP). Esta regla significa que cualquier origen puede acceder al ALB utilizando cualquier puerto, ¡lo cual es bastante abierto e inseguro!

4.5. Ahora, haz clic en **Editar reglas de entrada** y elimina la regla actual. Luego, agrega una nueva regla con las siguientes propiedades:

- `Rango de puertos: 80`
- `Origen: 0.0.0.0/0`

![image alt ><](./images/SecurityGroupRule.png)

Guarda los cambios.

Así que aunque todo internet puede tener acceso a tu ALB, el tráfico solo está permitido a través del puerto 80, por lo que si regresas a tu navegador y apuntas al puerto 90 (`<ALB-DNS>:90`) 
no deberías tener acceso a él.

*Nota*: Aún puedes ver una visualización del sitio web previamente accedido a través del puerto 90 y almacenado en caché. Pero si actualizas la página web, no se cargará.

Para verificar si la regla se creó correctamente, en tu navegador, apunta al puerto 80 (por defecto solo puedes poner `<ALB-DNS>` en la barra de búsqueda de tu navegador) 
y verifica si puedes ver el mensaje del Panel de control.

Al configurar los grupos de seguridad, restringiste el acceso a datos sensibles presentados en el puerto 90 del ALB a personal autorizado dentro de la empresa. Esto se hizo para mejorar la seguridad general del sistema, 
siguiendo las pautas del pilar de Seguridad del Marco Bien Arquitectado de AWS y el principio de "Priorizar la seguridad".


<div id='5'/>

## 5 - Verificación de disponibilidad de EC2

Si revisas la arquitectura dada de la aplicación web, notarás que cada instancia de EC2 pertenece a una **Zona de Disponibilidad (AZ)** diferente. Las Zonas de Disponibilidad son centros de datos aislados, cada uno con infraestructura de energía, refrigeración y redes independientes. Diseñar aplicaciones para abarcar múltiples AZs mejora su tolerancia a fallos y resiliencia, y proporciona una base para sistemas altamente disponibles. Esta práctica en la nube está relacionada con el pilar de confiabilidad de las soluciones basadas en la nube, así como con el principio de "Planificar para el fallo".


5.1. Para confiabilidad, nuevamente puedes poner el `Nombre DNS` en tu navegador. Pero ahora, echa un vistazo más de cerca al mensaje:

```
Datos del panel de control servidos desde el host <host-internal-ip>.ec2.internal en la zona de disponibilidad <AZ>
```

Verás una IP interna de uno de los servidores del ALB y una zona de disponibilidad. Actualiza la página y verás que tanto la IP interna como los valores de la zona de disponibilidad cambian. 
¡Eso significa que los datos se sirvieron cada vez desde diferentes zonas de disponibilidad e IP internas! ¡Imagina, que si algo sale mal con una de las AZ, tus datos aún pueden ser servidos 
desde otra!

Las implementaciones Multi-AZ en AWS ofrecen a las empresas una solución robusta para mitigar el impacto de posibles fallas. Al distribuir los componentes de la aplicación en diferentes AZ, 
las organizaciones pueden lograr alta disponibilidad y tolerancia a fallas. En caso de una interrupción o problema en una AZ, el tráfico se redirige automáticamente a instancias saludables 
en otras AZ, asegurando la entrega de servicios ininterrumpida. Este enfoque arquitectónico minimiza el tiempo de inactividad, mejora el rendimiento y contribuye a una experiencia de usuario fluida.

Si bien las implementaciones Multi-AZ mejoran la confiabilidad dentro de una sola región de AWS, adoptar una estrategia multi-región lleva la resiliencia al siguiente nivel. Las implementaciones 
multi-región implican replicar aplicaciones y datos en diferentes ubicaciones geográficas, ofreciendo protección contra desastres regionales, problemas geopolíticos o fallas de infraestructura. 
Este enfoque garantiza la continuidad del negocio a una escala más amplia y atiende a los usuarios globales, proporcionando acceso de baja latencia a los servicios.


<div id='6'/>

## 6 - Realizando Autoescalado

Necesitas asegurarte de que estás utilizando los recursos adecuadamente y de que estás adoptando el principio de "Diseñar para la escalabilidad" escalando hacia arriba y hacia abajo según la demanda.


<div id='6-1'/>

### 6.1 - Utilizando Recursos de Forma Eficiente

Las instancias actuales son del tipo `t3.micro` (*t* es la familia de la instancia, *3* es la generación y micro representa el tamaño de la instancia EC2).
Te diste cuenta de que esas instancias pueden ser un poco excesivas para tu infraestructura, así que después de leer un poco más sobre los [tipos de instancias](https://aws.amazon.com/ec2/instance-types/t3/) decidiste reducir su tamaño y utilizar una instancia `t3.nano` más adecuada.
Para escalar las instancias, necesitas modificar la forma en que se crean dentro del **Grupo de Autoescalado**, lo cual se puede hacer a través de las **Plantillas de Lanzamiento**.


6.1.1. En la consola de AWS, busca el servicio **EC2** y encuentra la sección de **Grupos de escalado automático** en el panel izquierdo. Selecciona el grupo de escalado automático proporcionado.


6.1.2. En la pestaña **Detalles**, encuentra la sección de **Plantilla de lanzamiento** y haz clic en **Editar**.

![imagen alt ><](./images/LaunchTemplateEdit.png)


6.1.3. Encuentra un menú desplegable de **Versión** y debajo un enlace para `Crear una nueva versión de la plantilla de lanzamiento` (debajo de él). Esto abrirá una página para crear una nueva versión de la plantilla.

![imagen alt ><](./images/CreateLaunchTemplateVersion.png)


6.1.4. Asegúrate de que la casilla de verificación **Proporcionar orientación para ayudarme a configurar una plantilla que pueda usar con EC2 Auto Scaling** esté **desmarcada**:

![image alt ><](./images/TemplateCheckbox.png)

En la sección **Tipo de instancia**, elige el tipo de instancia `t3.nano`:

![image alt ><](./images/InstanceType.png)

Desplázate hacia abajo hasta encontrar la **Configuración de red**. En la subred, haz clic en el menú desplegable y selecciona `No incluir en la plantilla de inicio`.

![image alt ><](./images/AutoscalingSubnet.png)

Luego, haz clic en ![](./images/CreateTemplateVersion.png).


6.1.5. Regrese a su grupo de escalado automático y luego vaya a **Detalles** > **Plantilla de lanzamiento** > **Editar**. En el menú desplegable **Versión**, ahora puede seleccionar la opción `Última` y hacer clic en el botón ![](./images/Update.png) en la parte inferior de la página.

*Nota*: Si no puede utilizar la nueva plantilla y ha recibido este error

![image alt ><](./images/TemplateError.png)

podría ser porque no eligió la instancia EC2 correcta (`t3.nano`) en el paso 6.1.4. 
En este caso, repita los pasos 6.1.3-6.1.4 y elija `t3.nano`. 
El número debería ser `3` y el tamaño debería ser `nano`.


6.1.6. Ahora, necesitas terminar las instancias para que las nuevas instancias que se lanzarán sean del tipo de instancia recién definido. En el servicio **EC2**, ve a **Instancias** y selecciona las dos que actualmente están en ejecución bajo el nombre de `de-c1w3-asg`. Haz clic derecho en cualquiera de ellas y selecciona **Terminar instancia** (o **Terminar (eliminar) instancia**). Es posible que necesites confirmar esta acción en la ventana emergente.

![image alt ><](./images/TerminateInstance.png)

Dado que el grupo de escalado automático tiene una capacidad deseada de 2 instancias, después de algunos minutos las nuevas instancias (con el tipo de instancia `t3.nano`) deberían comenzar a ejecutarse nuevamente. Actualiza la interfaz de usuario cada minuto hasta que aparezca como en ejecución. Antes de pasar a la siguiente sección del laboratorio, espera hasta que las instancias recién creadas tengan el estado de instancia `En ejecución`.


### 6.2 - Realización de Auto Scaling

6.2.1. Las instancias de EC2 en su grupo de Auto Scaling pueden aumentar o disminuir en número dependiendo de la demanda. Con el grupo de Auto Scaling, solo paga por lo que usa. Cuando la demanda disminuye, AWS Auto Scaling eliminará automáticamente cualquier EC2 adicional para evitar gastos excesivos.

Actualmente, Auto Scaling no está habilitado, por lo que el grupo de Auto Scaling no podrá escalar hacia arriba o hacia abajo. Para habilitar Auto Scaling, deberá crear una política de escalado donde necesitará especificar métricas y valores de umbral que invoquen el proceso de escalado.

En la consola de AWS, regrese a la sección de **Grupos de Auto Scaling** bajo el servicio de **EC2**, elija su grupo de Auto Scaling y haga clic en la pestaña de **Escalado automático**.

6.2.2. En la sección de **Políticas de escalado dinámico**, haga clic en **Crear política de escalado dinámico**.

![image alt ><](./images/DynamicScalingPolicy.png)


6.2.3. Crear una nueva política con las siguientes propiedades:

- Tipo de política: `Escalado de seguimiento de objetivo`
- Nombre de la política de escalado: `de-c1w3-scaling-policy`
- Tipo de métrica: `Recuento de solicitudes por objetivo del balanceador de carga de la aplicación`
- Grupo objetivo: `de-c1w3-ec2-tg-port80`
- Valor objetivo: `60`
- Tiempo de calentamiento de la instancia: `60` segundos

Haz clic en el botón **Crear**.

El significado de esos valores es:
- Dado que el tipo de métrica es un recuento de solicitudes por objetivo, con un valor objetivo de 60, significa que cuando el recuento de solicitudes HTTP (puerto 80) supera los 60 (valor elegido solo por simplicidad en este laboratorio), el grupo de escalado automático puede escalar para agregar más instancias para manejar la carga aumentada. Si el recuento de solicitudes por objetivo cae por debajo de 60, el grupo de escalado automático reducirá el número de instancias para ahorrar costos y mantener la eficiencia.

- Un tiempo de calentamiento se refiere a un período durante el cual las instancias recién lanzadas pueden recibir tráfico e inicializarse completamente antes de considerarse en servicio para las métricas de evaluación del escalado automático. Durante el tiempo de calentamiento, el grupo de escalado automático monitorea la salud y el estado de las instancias para asegurarse de que estén listas para manejar la carga de trabajo de producción. Por simplicidad en este laboratorio, se establece este valor en 60 segundos, pero el valor predeterminado de AWS es de 300 segundos (5 minutos).

6.2.4. Realiza una prueba más intensa con Apache Benchmark para probar la política de escalado que se creó. Ve a **CloudShell** y ejecuta el siguiente comando (reemplaza `ALB-DNS` con el `Nombre DNS` de tu ALB):

```
ab -n 1000000 -c 200 http://<ALB-DNS>/
```


Esta prueba debería ejecutarse durante algunos minutos, mientras tanto, puedes monitorear tus instancias.
Regresa al servicio de **Grupos de escalado automático** en la consola de AWS y haz clic en **Monitoreo** y luego en **EC2** para observar nuevamente las métricas de CPU y red.
Después de un tiempo (aproximadamente 5 min) cuando esas métricas comiencen a aumentar, puedes cambiar a la pestaña de **Actividad** y verás algunas notificaciones sobre instancias adicionales que se están lanzando para manejar el tráfico a través del puerto 80.

Después de que la prueba de estrés termine, puedes seguir monitoreando las métricas y la pestaña de **Actividad** para ver cuándo las métricas vuelven a bajar y incluso las instancias lanzadas son terminadas.
Ten en cuenta que hay cierto retraso en las gráficas de métricas en la pestaña de **Monitoreo** después de que inicias la prueba. Puedes volver a ejecutar el comando para la prueba si deseas mantener el estrés durante más tiempo.
